# TimeSeries Bing Your Own Model

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

---

## Runtime

This notebook takes approximately 30 minutes to run.

## Prerequisites

### Install Mercury

If not already installed, the following cell will install the `mercury` package in order to display the `analysis_config.json` and explainability job output within the notebook.

In [1]:
!pip install mercury -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.31.7 requires botocore==1.33.7, but you have botocore 1.29.165 which is incompatible.
awscli 1.31.7 requires s3transfer<0.9.0,>=0.8.0, but you have s3transfer 0.6.2 which is incompatible.
sagemaker 2.214.3 requires boto3<2.0,>=1.33.3, but you have boto3 1.26.83 which is incompatible.


## Install Development SDK

As of 13-12-23, time series explainability (TSX) is in closed beta. The changes to the SDK to support it are not merged into the `sagemaker-python-sdk` repository. Instead, we install a version of the PySDK that does support these changes directly via a wheel file. 

In [2]:
!pip uninstall sagemaker -y -q c
!pip install sagemaker
!pip show sagemaker

  Using cached sagemaker-2.214.3-py3-none-any.whl.metadata (14 kB)
  Using cached s3transfer-0.10.1-py3-none-any.whl.metadata (1.7 kB)
Using cached sagemaker-2.214.3-py3-none-any.whl (1.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 105.4 MB/s eta 0:00:00m eta 0:00:016m0:00:01
Using cached s3transfer-0.10.1-py3-none-any.whl (82 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.165
    Uninstalling botocore-1.29.165:
      Successfully uninstalled botocore-1.29.165
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.6.2
    Uninstalling s3transfer-0.6.2:
      Successfully uninstalled s3transfer-0.6.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.83
    Uninstalling boto3-1.26.83:
      Successfully uninstalled boto3-1.26.83
ERROR: pip's dependency resolver does not currently take into account all the p

### Import Libraries

The model used in this example notebook is DeepAr forecasting model in AWS. For more information, please check https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html (TODO, change this)

Besides, a separate notebook will introduce how to bring your own time series model into TSX.

In [3]:
import yaml
import pandas as pd
import numpy as np
import sagemaker
import boto3
import json
import mercury
import pprint

from sagemaker import get_execution_role

session = boto3.Session()
s3_client = session.client("s3")
sagemaker_session = sagemaker.Session()
s3_bucket = sagemaker.Session().default_bucket()
region = session.region_name
sagemaker_client = boto3.client("sagemaker", region_name=region)
sagemaker_role = get_execution_role()

endpoint_name = "timeseries-byom-endpoint"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/zicanl/.config/sagemaker/config.yaml


Compress the model package

In [4]:
!cd model && tar -czvf ../model.tar.gz code/ model.pth

code/
code/inference.py
code/requirements.txt
model.pth


Upload model to S3. The example model is the linear regression model from [darts](https://unit8co.github.io/darts/generated_api/darts.models.forecasting.linear_regression_model.html#linear-regression-model)

In [10]:
bucket_prefix = "linear_ts"
model_file_path = "model.tar.gz"
model_s3_suffix = f"{bucket_prefix}/" + model_file_path

s3_client.upload_file(model_file_path, s3_bucket, model_s3_suffix)

In [13]:
from sagemaker.pytorch.model import PyTorchModel

sagemaker_model = PyTorchModel(
    model_data=f"s3://{s3_bucket}/{model_s3_suffix}",  # specify S3 location of model.tar.gz
    entry_point="inference.py",  # specify entry point
    framework_version="1.10.2",
    py_version="py38",
    role=sagemaker_role,
)

# deploy model to SageMaker Inference
predictor = sagemaker_model.deploy(
    initial_instance_count=1,  # number of instances
    instance_type="ml.m5.4xlarge",  # ec2 instance type
    endpoint_name=endpoint_name,
)

-----------!

### Verify the endpoint

In [14]:
input_instances = {
    "target": [1008.89, 1008.76, 1008.66],
    "start": "2020-01-01 16:20:00",
    "dynamic_feat": [
        [0.23, 0.21, 0.19, 0.16, 0.13, 0.08, 0.0, 0.0, 0.0],
        [0.71, 0.75, 0.73, 0.37, 0.33, 0.34, 0.19, 0.03, 0.11],
    ],
}
predictor_input = {
    "instances": input_instances,
}
predictor_input

{'instances': {'target': [1008.89, 1008.76, 1008.66],
  'start': '2020-01-01 16:20:00',
  'dynamic_feat': [[0.23, 0.21, 0.19, 0.16, 0.13, 0.08, 0.0, 0.0, 0.0],
   [0.71, 0.75, 0.73, 0.37, 0.33, 0.34, 0.19, 0.03, 0.11]]}}

In [15]:
from sagemaker.serializers import JSONSerializer

predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session, serializer=JSONSerializer()
)
prediction = predictor.predict(predictor_input)
print(json.loads(prediction))

{'predictions': {'mean': [1008.5656193655171, 1008.511339974418, 1008.4186285678663, 1008.3432658290359, 1008.2426915722609, 1008.1615810491433, 1008.1274301835906, 1008.0939011448609, 1008.0749872974072, 1008.0284006424769]}}


## Time Series Explainability

### Import Components

Import the components needed to make a TSX call.

In [16]:
from sagemaker.clarify import (
    AsymmetricShapleyValueConfig,  # config for the explainability algorithm
    DataConfig,  # general-purpose DataConfig. time series-specific data config object is provided to this
    ModelConfig,  # general-purpose ModelConfig. time series-specific data config object is provided to this
    SageMakerClarifyProcessor,  # processor object, the job call is made via this
    TimeSeriesDataConfig,  # time series-specific data config object
    TimeSeriesModelConfig,  # time series-specific predictor config object
    TimeSeriesJSONDatasetFormat,  # time series-specific dataset format
)

### Set Configurations

In [17]:
dataset_name = "time_series_byom_mock_data.json"

# Content template
c_template = '{"instances": $record}'
# Record template
r_template = (
    '{"start": $start_time, "target": $target_time_series, "dynamic_feat": $related_time_series}'
)

s3 = boto3.client("s3")  # s3 client
bucket_name = sagemaker.Session().default_bucket()
bucket_uri = "s3://" + bucket_name + "/byom/"

s3_client.upload_file(dataset_name, bucket_name, f"byom/data/{dataset_name}")

### Asymmetric Shapley value

Our time series forecasting explainability algorithm hinges on the application of the asymmetric Shapley values (ASV) from the theory of cooperative games. The ASV is a modification of the well-known Shapley value (e.g SHAP) that discards the symmetry axiom, but retains the efficiency exioms (i.e. attributions sum up to the predictions). Coalitions of features are generated based on a given probability distribution over feature *permutations* (rather than over *subsets* in the case of the Shapley value). In the case of time series, the distributions we use puts zero probability on permutations of features that do not respect the temporal dependencies, i.e. that have "holes". 

**References:**
- Our main scientific reference is https://arxiv.org/abs/1910.06358. We scale the approach of the paper to include also static covariates, related time series and implement a stochastic estimator for efficiency.
- A very useful math reference is [Probabilistic values by RJ Weber](http://www.library.fa.ru/files/Roth2.pdf#page=109);  specifically, section 8 about random-order values (these are the same mathematical construction of ASV). 

### Create `AsymmetricShapleyValueConfig`

An `AsymmetricShapleyValueConfig` is used to configure the algorithm Clarify uses for time series explainability. It takes the following arguments:

- `direction`: direction of explanation to be used. Available explanation types are `"chronological"`, `"anti_chronological"`, `"bidirectional"`. The cronological direction highlights the effect of older timesteps over more recent one, while the anti-chronological direction higlights the effect of timesteps closer to the forecasting. Bidirectional is a combination of the previous two modes. 
- `granularity`: Granularity of explanation to be used. Available granularities are `"timewise"` and `"fine_grained"`. The first granularity is fast and computes the attribution of individual timesteps toward the forecast, not making distinctions of related time series. The fine-grained mode is slower, but computes an attribution for every timestep and every feature dynamic, distinguishing between related and target TS.
- `num_samples`: Number of samples to be used in the Asymmetric Shapley Value forecasting algorithm. Only applicable when using `"fine_grained"`  explanations. This represents the number of permutations sampled for computing the ASV. 

The notebook sets `explanation_direction` and `granularity` as variables for later reference.

In [18]:
direction = "chronological"
granularity = "fine_grained"

Only then does the notebook create the `AsymmetricShapleyValueConfig` object.

In [19]:
asym_shap_val_config = AsymmetricShapleyValueConfig(
    direction=direction,
    granularity=granularity,
    num_samples=2,  # (dimension of target_time_series + dimension of related_time_series) ^ 2
    baseline={
        "target_time_series": "zero",
        "related_time_series": "zero",
    },
)

### Create `TimeSeriesDataConfig`

A `TimeSeriesDataConfig` object is used to configure data I/O settings specific to TSX. It takes the following arguments:

- `target_time_series`: A string or a zero-based integer index. Used to locate the target time series in the shared input dataset. If this parameter is a string, then all other parameters must also be strings or lists of strings. If this parameter is an int, then all others must be ints or lists of ints.
- `item_id`: A string or a zero-based integer index. Used to locate item id in the shared input dataset.
- `timestamp`: A string or a zero-based integer index. Used to locate timestamp in the shared input dataset.
- `related_time_series`: Optional. An array of strings or array of zero-based integer indices. Used to locate all related time series in the shared input dataset (if present).
- `static_covariates`: Optional. An array of strings or array of zero-based integer indices. Used to locate all item metadata fields in the shared input dataset (if present).
- `dataset_format`: Optional. A string which describes the format of the data files provided for analysis. Should only be provided when dataset is in JSON format. Currently, we support `columns` and `timestamp_records` where example mock data files `ts_cols.json` and `time_series_mock_data.json` are provided respectively.

This `TimeSeriesDataConfig` helps the container to parse the data needed for the analysis. Any additional data columns will be excluded if not providing corresponding Jmes_path to locate them.

In [20]:
ts_data_config = TimeSeriesDataConfig(
    target_time_series="[].p_mbar",
    item_id="[].item_id",
    timestamp="[].timestamp",
    related_time_series=["[].rain_mm", "[].T_degC"],
    dataset_format=TimeSeriesJSONDatasetFormat.TIMESTAMP_RECORDS,
)

### Create `TimeSeriesModelConfig`

A `TimeSeriesModelConfig` is used to configure model settings specific to TSX. At the moment it has only one argument:

- `forecast`: JMESPath expression to extract the forecast result.

In [21]:
ts_model_config = TimeSeriesModelConfig(
    forecast="predictions.mean",
)

### Create DataConfig

General information about the dataset the TimeSeries model uses is provided to `DataConfig`. Here, we are providing where to retrieve the dataset, where to output the explainability job results, what format the dataset is in, and our TSX specific data settings.

In [22]:
input_uri = bucket_uri + "data/" + dataset_name
output_path = bucket_uri + "output_byom"

data_config = DataConfig(
    s3_data_input_path=input_uri,
    s3_output_path=output_path,
    dataset_type="application/json",
    time_series_data_config=ts_data_config,
    headers=["item_id", "timestamp", "p_mbar", "rain_mm", "T_degC"],
)

's3://sagemaker-us-west-2-678264136642/byom/data/time_series_byom_mock_data.json'

### Create ModelConfig

With `ModelConfig` is configured here, Clarify will deploy the specified model to a new endpoint.

In [23]:
model_config = ModelConfig(
    endpoint_name=endpoint_name,
    content_type="application/json",
    accept_type="application/json",
    content_template=c_template,
    record_template=r_template,
    time_series_model_config=ts_model_config,
)

It is also possible to specify an existing endpoint for Clarify to use with the following modifications to the `ModifyConfig` call:

1. Omitting `model_name`, `instance_count`, `instance_type`, and `endpoint_name_prefix`.
2. Provided `endpoint_name`.

### Setup Processor

Create the `Processor` object that will setup the explainability job.

In [24]:
instance_count = 1
instance_type = "ml.c5.2xlarge"

clarify_processor = SageMakerClarifyProcessor(
    role=sagemaker_role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    job_name_prefix="clarify-tsx-byom-job-demo",
)

### Run Explainability Call

In [25]:
clarify_processor.run_explainability(
    data_config=data_config,
    model_config=model_config,
    explainability_config=asym_shap_val_config,
)

INFO:sagemaker:Creating processing-job with name clarify-tsx-byom-job-demo-2024-04-09-17-54-47-904


INFO:sagemaker-clarify-processing:Starting SageMaker Clarify Processing job
INFO:analyzer.data_loading.data_loader_util:Analysis config path: /opt/ml/processing/input/config/analysis_config.json
INFO:analyzer.data_loading.data_loader_util:Analysis result path: /opt/ml/processing/output
INFO:analyzer.data_loading.data_loader_util:This host is algo-1.
INFO:analyzer.data_loading.data_loader_util:This host is the leader.
INFO:analyzer.data_loading.data_loader_util:Number of hosts in the cluster is 1.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 18:00:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
#015[Stage 0:>                                                          (0 + 8) / 8]#015#015                                                                                #015#015[Stage 4:====================================

## Analysis Config

### Retrieve Config From s3

In [26]:
s3.download_file(bucket_name, "byom/output_byom/analysis_config.json", "analysis_config.json")

### Display Config

In [27]:
with open("./analysis_config.json", "r") as analyis_config_file:
    analysis_config = json.load(analyis_config_file)
    # mercury.JSON(analysis_config, level = 3)
    config_printer = pprint.PrettyPrinter(width=200, compact=False)
    config_printer.pprint(analysis_config)

{'dataset_type': 'application/json',
 'headers': ['item_id', 'timestamp', 'p_mbar', 'rain_mm', 'T_degC'],
 'methods': {'asymmetric_shapley_value': {'baseline': {'related_time_series': 'zero', 'target_time_series': 'zero'}, 'direction': 'chronological', 'granularity': 'fine_grained', 'num_samples': 2},
             'report': {'name': 'report', 'title': 'Analysis Report'}},
 'predictor': {'accept_type': 'application/json',
               'content_template': '{"instances": $record}',
               'content_type': 'application/json',
               'endpoint_name': 'timeseries-byom-endpoint',
               'record_template': '{"start": $start_time, "target": $target_time_series, "dynamic_feat": $related_time_series}',
               'time_series_predictor_config': {'forecast': 'predictions.mean'}},
 'time_series_data_config': {'dataset_format': 'timestamp_records',
                             'item_id': '[].item_id',
                             'related_time_series': ['[].rain_mm', '[]

## Explainability Results

### Retrieve Results From s3

In [28]:
full_result_path = f"byom/output_byom/asymmetric_shapley_value/{granularity}_{direction}/out.jsonl"

s3.download_file(bucket_name, full_result_path, "results.jsonl")

### Display Results

In [30]:
with open("./results.jsonl", "r") as results_file:
    results_lines = results_file.readlines()
    explainability_results = [json.loads(jsonline) for jsonline in results_lines]
    mercury.JSON(explainability_results, level=5)
    # results_printer = pprint.PrettyPrinter(width = 200, depth = 5, compact = False)
    # results_printer.pprint(explainability_results)

## Clean Up

Remove downloaded/installed files and deployed resources as necessary.

In [31]:
# remove the model, endpoint_config and endpoint from sagemaker
boto3.client("sagemaker").delete_endpoint_config(EndpointConfigName=endpoint_name)
boto3.client("sagemaker").delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'ee756241-e28a-4dfb-ae34-42ca6f0e270f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ee756241-e28a-4dfb-ae34-42ca6f0e270f',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 09 Apr 2024 18:01:10 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-clarify|time_series_byom|time_series_bring_your_own_model.ipynb)